# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output/cities.csv")
cities_df.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ancud,-41.8697,-73.8203,42.22,75,13,4.59,CL,1660779196
1,1,Jamestown,42.0970,-79.2353,71.19,82,0,0.00,US,1660778816
2,2,Kutulik,53.3508,102.7839,50.14,80,99,1.52,RU,1660779197
3,3,Vao,-22.6667,167.4833,61.95,90,100,17.45,NC,1660779198
4,4,Yeppoon,-23.1333,150.7333,60.58,45,0,0.98,AU,1660779198


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
locations = cities_df[['Latitude', 'Longitude']].astype(float)
weights = cities_df['Humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                    dissipating=False, max_intensity=cities_df['Humidity'].max() ,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# filter cities with ideal weather conditions
ideal_weather_df = cities_df.loc[
    (cities_df['Max Temp']>70) &
    (cities_df['Max Temp']<80) &
    (cities_df['Cloudiness']<15) &
    (cities_df['Wind Speed']<8) &
    (cities_df['Humidity']<45) &
    (cities_df['Humidity']<65)
].dropna().reset_index(drop=True)
print(len(ideal_weather_df))

9


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
from csv import excel_tab


hotel_df = pd.DataFrame()
hotel_df['Hotel Name'] = ''
hotel_df['Lat'] = ''
hotel_df['Lng'] = ''
hotel_df['City'] = ''
hotel_df['Country'] = ''
params = {
    'radius' : 5000,
    'keyword' : 'hotel',
    'key' : g_key,
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in ideal_weather_df.iterrows():
    params['location'] = f"{row['Latitude']}, {row['Longitude']}"
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        hotel_df.loc[index, 'Lat'] = response['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = response['results'][0]['geometry']['location']['lng']
        hotel_df.loc[index, 'City'] = row['City']
        hotel_df.loc[index, 'Country'] = row['Country']
    except:
        continue

hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Diamante Rosa Palace Hotel,-18.47487,-47.188763,Coromandel,BR
1,Club & Hotel Letoonia,36.634879,29.099307,Fethiye,TR
2,Hotel Camapuã,-9.827301,-45.336107,Gilbués,BR
3,Dumont Palace Hotel,-6.640577,-51.980267,São Félix do Xingu,BR
4,Hotel Star,-14.533978,-49.959249,Crixás,BR
5,Hotel Diniz,-5.498093,-45.252747,Barra do Corda,BR
6,Hotel D'Borges,-17.291719,-49.022404,Piracanjuba,BR
8,TANTANA HOTEL FERGANA,40.396798,71.765271,Yangi Marg`ilon,UZ


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))